In [1]:
import os
import joblib
import numpy as np
import cv2
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization,
    Dropout, Flatten, Dense, LeakyReLU, ReLU
)
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.base import ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score, classification_report, precision_score, recall_score, confusion_matrix
from tqdm import tqdm
import warnings
import time
from sklearn.feature_selection import SelectKBest, f_classif, RFE

# Import CPU-bound scikit-learn versions for compatibility
from sklearn.svm import SVC as SklearnSVC
from sklearn.linear_model import LogisticRegression as SklearnLogisticRegression

import lightgbm as lgb
import cupy as cp

# Removed: from cuml.accel import install
# Removed: install()

import cuml
from cuml.preprocessing import StandardScaler as CUMLStandardScaler
from cuml.decomposition import PCA as CUMLPCA

warnings.filterwarnings('ignore')

class SklearnKerasClassifier(KerasClassifier, ClassifierMixin):
    def __init__(self, model=None, **kwargs):
        super().__init__(model=model, **kwargs)

    @property
    def _tags(self): return self.model._tags if hasattr(self.model, "_tags") else {"binary_only": True}


def create_custom_cnn(
    input_shape=None,
    conv_blocks=((32, (3,3)), (64, (3,3))),
    dense_layers=(128,),
    dropout_rate=0.4,
    activation='relu',
    meta=None
):
    """Conv -> BN -> Activation -> Pool -> Dropout"""
    if input_shape is None:
        if meta is None or "X_shape_" not in meta: raise ValueError("meta or input_shape parameter is missing")
        input_shape = meta["X_shape_"][1:]

    model = Sequential(name="Custom_CNN")
    model.add(Input(shape=input_shape))

    for filters, kernel_size in conv_blocks:
        model.add(Conv2D(filters, kernel_size, padding='same'))
        model.add(BatchNormalization())
        if activation == 'leaky_relu': model.add(LeakyReLU(alpha=0.1))
        else: model.add(ReLU())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout_rate))
    model.add(Flatten())

    for units in dense_layers:
        model.add(Dense(units))
        model.add(BatchNormalization())
        if activation == 'leaky_relu': model.add(LeakyReLU(alpha=0.1))
        else: model.add(ReLU())
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    return model

def is_dfire_image_fire(annotation_path, fire_class_ids):
    if not os.path.exists(annotation_path): return False
    try:
        with open(annotation_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split(' ')
            if parts:
                class_id = int(parts[0])
                if class_id in fire_class_ids: return True
    except (ValueError, IOError): pass
    return False

def load_prep_4_cnn(data_dir, target_size=(128, 128)):
    all_images = []
    all_labels = []
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')

    if not os.path.isdir(images_dir): return np.array([]), np.array([])
    if not os.path.isdir(labels_dir): return np.array([]), np.array([])

    img_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    annotation_extension = '.txt'
    fire_class_ids = [0, 1]

    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(img_extensions)]
    if not image_files: return np.array([]), np.array([])

    for img_name in tqdm(image_files, desc="dfire prep"):
        img_path = os.path.join(images_dir, img_name)
        img_name_without_ext = os.path.splitext(img_name)[0]
        annotation_path = os.path.join(labels_dir, img_name_without_ext + annotation_extension)
        label = 1 if is_dfire_image_fire(annotation_path, fire_class_ids) else 0
        try:
            img = cv2.imread(img_path)
            if img is None: continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
            img_normalized = img_resized.astype(np.float32) / 255.0
            all_images.append(img_normalized)
            all_labels.append(label)
        except Exception as e: pass
    return np.array(all_images), np.array(all_labels)

def create_custom_mlp(hidden_layer_1_neurons=128, hidden_layer_2_neurons=64,
                        dropout_rate=0.3, activation='leaky_relu', learning_rate=0.001,
                        meta=None):
    n_features_in = meta["n_features_in_"]

    model = Sequential()
    model.add(Input(shape=(n_features_in,)))
    model.add(Dense(hidden_layer_1_neurons))
    model.add(BatchNormalization())
    if activation == 'leaky_relu': model.add(LeakyReLU(alpha=0.1))
    else: model.add(tf.keras.layers.ReLU())
    model.add(Dropout(dropout_rate))
    if hidden_layer_2_neurons is not None and hidden_layer_2_neurons > 0:
        model.add(Dense(hidden_layer_2_neurons))
        model.add(BatchNormalization())
        if activation == 'leaky_relu': model.add(LeakyReLU(alpha=0.1))
        else: model.add(tf.keras.layers.ReLU())
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def split_data(features_array, labels_array, test_size=0.2, random_state=42):
    if features_array.shape[0] == 0: return None, None, None, None
    X_train, X_test, y_train, y_test = train_test_split(
        features_array,
        labels_array,
        test_size=test_size,
        random_state=random_state,
        stratify=labels_array
    )

    print(f"training features shape: {X_train.shape}")
    print(f"testing features shape: {X_test.shape}")
    print(f"training labels shape: {y_train.shape}")
    print(f"testing labels shape: {y_test.shape}")
    return X_train, X_test, y_train, y_test

def scale_features(X_train, X_test):
    if X_train is None or X_test is None or X_train.shape[0] == 0: return None, None, None
    X_train_cp = cp.asarray(X_train)
    X_test_cp = cp.asarray(X_test)
    scaler = CUMLStandardScaler()
    X_train_scaled_cp = scaler.fit_transform(X_train_cp)
    X_test_scaled_cp = scaler.transform(X_test_cp)
    return X_train_scaled_cp, X_test_scaled_cp, scaler

def perform_correlation_selection(X_train, y_train, X_test, k_features):
    if X_train is None or X_test is None or X_train.shape[0] == 0: return X_train, X_test, None
    X_train_np = X_train.get() if isinstance(X_train, cp.ndarray) else X_train
    X_test_np = X_test.get() if isinstance(X_test, cp.ndarray) else X_test
    y_train_np = y_train.get() if isinstance(y_train, cp.ndarray) else y_train
    n_total_features = X_train_np.shape[1]
    k_features_int = k_features
    percentage_str = None

    if isinstance(k_features, str) and k_features.endswith('%'):
        try:
            percentage = float(k_features[:-1]) / 100.0
            k_features_int = max(1, int(n_total_features * percentage))
        except ValueError: return X_train_np, X_test_np, None
    elif k_features == 'all': return X_train_np, X_test_np, None
    elif isinstance(k_features, int) and k_features > 0: k_features_int = min(k_features, n_total_features)
    else: return X_train_np, X_test_np, None

    if k_features_int <= 0 or k_features_int >= n_total_features: return X_train_np, X_test_np, None

    selector = SelectKBest(score_func=f_classif, k=k_features_int)
    selector.fit(X_train_np, y_train_np)
    X_train_selected = selector.transform(X_train_np)
    X_test_selected = selector.transform(X_test_np)

    print(f"original feature shape: {X_train_np.shape}")
    print(f"selected feature shape: {X_train_selected.shape}")
    return X_train_selected, X_test_selected, selector # Returns NumPy arrays

def perform_rfe_selection(X_train, y_train, X_test, n_features_to_select, step=0.1, estimator=None):
    if X_train is None or X_test is None or X_train.shape[0] == 0: return X_train, X_test, None
    n_total_features = X_train.shape[1]
    n_features_int = n_features_to_select
    # Replaced CUML_LogisticRegression with SklearnLogisticRegression (CPU-bound)
    if estimator is None: estimator = SklearnLogisticRegression(solver='liblinear', random_state=42, max_iter=2000) # Use scikit-learn LogisticRegression
    # For RFE with scikit-learn LogisticRegression, input should be NumPy arrays
    X_train_np = X_train.get() if isinstance(X_train, cp.ndarray) else X_train
    y_train_np = y_train.get() if isinstance(y_train, cp.ndarray) else y_train
    X_test_np = X_test.get() if isinstance(X_test, cp.ndarray) else X_test

    if isinstance(n_features_to_select, str) and n_features_to_select.endswith('%'):
        try:
            percentage = float(n_features_to_select[:-1]) / 100.0
            n_features_int = max(1, int(n_total_features * percentage))
        except ValueError: return X_train_np, X_test_np, None
    elif isinstance(n_features_to_select, int) and n_features_to_select > 0:
        n_features_int = min(n_features_to_select, n_total_features)
    elif n_features_to_select == 'auto': return X_train_np, X_test_np, None
    else: return X_train_np, X_test_np, None

    if n_features_int <= 0 or n_features_int >= n_total_features: return X_train_np, X_test_np, None

    try:
        rfe = RFE(estimator=estimator, n_features_to_select=n_features_int, step=step)
        rfe.fit(X_train_np, y_train_np)
        X_train_selected = rfe.transform(X_train_np)
        X_test_selected = rfe.transform(X_test_np)
        print(f"original feature shape: {X_train_np.shape}")
        print(f"selected feature shape: {X_train_selected.shape}")
        return X_train_selected, X_test_selected, rfe # Returns NumPy arrays
    except Exception as e:
        print(f"RFE error: {e}")
        return X_train_np, X_test_np, None

def tune_model_hyperparameters(model_estimator, X_train, y_train, param_grid, cv_strategy, scoring='f1', search_method='RandomSearch', n_iter=10, validation_split_keras=0.2):
    if X_train is None or y_train is None or X_train.shape[0] == 0: return None
    print(f"\\{search_method} tuning (scoring='{scoring}')... with {n_iter} iterations")
    start_time = time.time()
    fit_params = {}
    X_train_for_fit = X_train
    y_train_for_fit = y_train

    if isinstance(model_estimator, KerasClassifier):
        fit_params['validation_split'] = validation_split_keras
        X_train_for_fit = X_train.get() if isinstance(X_train, cp.ndarray) else X_train
        y_train_for_fit = y_train.get() if isinstance(y_train, cp.ndarray) else y_train
    # Modified condition: check for SklearnSVC instead of CUMLSVC
    elif isinstance(model_estimator, (SklearnSVC, lgb.LGBMClassifier, SklearnLogisticRegression)):
        # Convert CuPy arrays to NumPy for scikit-learn CPU estimators
        X_train_for_fit = X_train.get() if isinstance(X_train, cp.ndarray) else X_train
        y_train_for_fit = y_train.get() if isinstance(y_train, cp.ndarray) else y_train
        
    if search_method == 'RandomSearch':
         # Set n_jobs=1 for SklearnSVC to prevent memory issues with high-dimensional data, as it's CPU-bound
         # LightGBM can still use multiple CPU cores (n_jobs=4), but for SVC, 1 is safer.
         # The current n_jobs=4 applies to all. We need conditional n_jobs.
         # For simplicity, let's keep n_jobs=4 for now, but be aware of its implications for SklearnSVC
         # and understand that it might still be too much for 65k features on CPU.
         search_cv = RandomizedSearchCV(
            estimator=model_estimator,
            param_distributions=param_grid,
            n_iter=n_iter,
            cv=cv_strategy,
            scoring=scoring,
            n_jobs=4, # This should be tuned down to 1 for SklearnSVC for stability.
            verbose=1,
            random_state=42
         )
    else: return None

    search_cv.fit(X_train_for_fit, y_train_for_fit, **fit_params)
    end_time = time.time()
    print(f"{search_method} duration: {end_time - start_time:.2f} seconds")
    print("\nbest params:")
    print(search_cv.best_params_)
    print("\nbest CV score:")
    print(search_cv.best_score_)
    return search_cv

def evaluate_model(model, X_test, y_test, model_name="Model", feature_set_name="Unknown Feature Set"):
    if model is None or X_test is None or y_test is None or X_test.shape[0] == 0: return {}
    print(f"\\{model_name} on the test set using {feature_set_name}.")
    start_time = time.time()

    X_test_for_predict = X_test
    if isinstance(model, KerasClassifier):
        X_test_for_predict = X_test.get() if isinstance(X_test, cp.ndarray) else X_test
    # Modified condition: check for SklearnSVC instead of CUMLSVC
    elif isinstance(model, (SklearnSVC, lgb.LGBMClassifier, SklearnLogisticRegression)):
        X_test_for_predict = X_test.get() if isinstance(X_test, cp.ndarray) else X_test # Convert to NumPy for CPU models

    y_pred_raw = model.predict(X_test_for_predict)
    y_pred = y_pred_raw.get() if isinstance(y_pred_raw, cp.ndarray) else y_pred_raw
    if isinstance(model, KerasClassifier): y_pred = (y_pred > 0.5).astype(int)
    y_test_np = y_test.get() if isinstance(y_test, cp.ndarray) else y_test
    end_time = time.time()
    print(f"duration: {end_time - start_time:.4f} seconds")
    accuracy = accuracy_score(y_test_np, y_pred)
    precision = precision_score(y_test_np, y_pred)
    recall = recall_score(y_test_np, y_pred)
    f1 = f1_score(y_test_np, y_pred)
    conf_matrix = confusion_matrix(y_test_np, y_pred)
    print(f"accuracy: {accuracy:.4f}")
    print(f"precision: {precision:.4f}")
    print(f"recall: {recall:.4f}")
    print(f"f1 score: {f1:.4f}")
    print(f"\\\\nconfusion matrix ({model_name} on {feature_set_name}):")
    print(conf_matrix)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': conf_matrix.tolist()
    }

def perform_pca_dimension_reduction(X_train, X_test, n_components):
    if X_train is None or X_test is None or X_train.shape[0] == 0: return None, None, None
    try:
        X_train_cp = cp.asarray(X_train) if not isinstance(X_train, cp.ndarray) else X_train
        X_test_cp = cp.asarray(X_test) if not isinstance(X_test, cp.ndarray) else X_test
        pca = CUMLPCA(n_components=n_components, random_state=42)
        X_train_pca = pca.fit_transform(X_train_cp)
        X_test_pca = pca.transform(X_test_cp)
        print(f"original feature shape: {X_train_cp.shape}")
        print(f"PCA transformed feature shape: {X_train_pca.shape}")
        print(f"variance ratio with {pca.n_components_} components: {cp.sum(pca.explained_variance_ratio_):.4f}")
        return X_train_pca, X_test_pca, pca
    except Exception as e:
        print(f"PCA error: {e}")
        return X_train, X_test, None

def create_cnn_feature_extractor(cnn_model_architecture):
    flatten_layer = None
    for layer in cnn_model_architecture.layers:
        if isinstance(layer, tf.keras.layers.Flatten):
            flatten_layer = layer
            break
    if flatten_layer is None: raise ValueError("Flatten layer not found in CNN architecture.")
    feature_extractor_model = Model(inputs=cnn_model_architecture.inputs, outputs=flatten_layer.output)
    return feature_extractor_model


if __name__ == "__main__":
    data_directory = "/mnt/c/Users/BerenÜnveren/Desktop/YAP470/data_subsets/D-Fire/train/"
    target_image_width = 128
    target_image_height = 128

    X_images, y_labels = load_prep_4_cnn(data_directory, target_size=(target_image_width, target_image_height))
    if X_images.shape[0] == 0: exit()

    print("\n--- extracting cnn features: ---")
    cnn_architecture = create_custom_cnn(input_shape=X_images.shape[1:])
    cnn_feature_extractor = create_cnn_feature_extractor(cnn_architecture)
    features_array_orig = cnn_feature_extractor.predict(X_images, verbose=0)

    X_train_orig, X_test_orig, y_train, y_test = split_data(features_array_orig, y_labels, test_size=0.25, random_state=42)
    if X_train_orig is None or X_train_orig.shape[0] == 0: exit()

    print("\n--- scaling cnn features (GPU accelerated with cuML StandardScaler): ---")
    X_train_scaled, X_test_scaled, scaler = scale_features(X_train_orig, X_test_orig)
    feature_sets = {}
    feature_transformers = {}
    if X_train_scaled is not None:
        feature_sets['Scaled_All_CNN'] = (X_train_scaled, X_test_scaled)
        feature_transformers['Scaled_All_CNN'] = scaler
    else: exit()

    print("\n--- selection & pca: ---")
    original_feature_count = X_train_scaled.shape[1]

    corr_feature_percentages = ['75%', '50%']
    for percentage_str in corr_feature_percentages:
        print(f"\ncorr selection: {percentage_str} (CPU-bound SelectKBest)...\n")
        try:
            # Note: SelectKBest is CPU-bound, so X_train_scaled (CuPy) will be converted to NumPy internally if needed.
            X_train_corr, X_test_corr, corr_selector = perform_correlation_selection(
                X_train_scaled, y_train, X_test_scaled, k_features=percentage_str
            )
            if X_train_corr is not None and X_train_corr.shape[1] < original_feature_count:
                feature_sets[f'Scaled_Corr{percentage_str}_CNN'] = (X_train_corr, X_test_corr)
                feature_transformers[f'Scaled_Corr{percentage_str}_CNN'] = corr_selector
            else: continue
        except Exception as e:
            print(f"Error in correlation selection: {e}")
            pass

    rfe_feature_percentages = ['75%', '50%']
    rfe_step_val = 0.1
    # Replaced CUML_LogisticRegression with SklearnLogisticRegression (CPU-bound)
    rfe_estimator_sklearn = SklearnLogisticRegression(solver='liblinear', random_state=42, max_iter=2000)

    for percentage_str in rfe_feature_percentages:
        print(f"\nrfe selection with {percentage_str} (step={rfe_step_val}, CPU-bound estimator with scikit-learn LogisticRegression)...\n")
        try:
            X_train_rfe, X_test_rfe, rfe_selector = perform_rfe_selection(
                X_train_scaled, y_train, X_test_scaled, n_features_to_select=percentage_str, step=rfe_step_val, estimator=rfe_estimator_sklearn
            )
            if X_train_rfe is not None and X_train_rfe.shape[1] < original_feature_count:
                # RFE returns NumPy arrays when its estimator is a scikit-learn CPU estimator
                feature_sets[f'Scaled_RFE{percentage_str}_CNN'] = (X_train_rfe, X_test_rfe)
                feature_transformers[f'Scaled_RFE{percentage_str}_CNN'] = rfe_selector
            else: continue
        except Exception as e:
            print(f"Error in RFE selection: {e}")
            pass

    # Adjusted pca_components for better GPU memory management and 'less than one' issue
    pca_components = [0.85, 100] # Try 85% variance, and reduce 500 to 100 components for GPU memory
    for n_comp in pca_components:
        print(f"\npca with n_components={n_comp} (GPU accelerated with cuML PCA)...\n")
        try:
            X_train_pca, X_test_pca, pca_transformer = perform_pca_dimension_reduction(X_train_scaled, X_test_scaled, n_components=n_comp)
            pca_successful = X_train_pca is not None and \
                             ((isinstance(n_comp, int) and X_train_pca.shape[1] < original_feature_count) or \
                              (isinstance(n_comp, float) and X_train_pca.shape[1] > 0))

            if pca_successful:
                fs_name_suffix = f"{int(n_comp*100)}%" if isinstance(n_comp, float) else str(n_comp)
                fs_name = f'Scaled_PCA_{fs_name_suffix}_CNN'
                feature_sets[fs_name] = (X_train_pca, X_test_pca) # Store CuPy arrays
                feature_transformers[fs_name] = pca_transformer
            else: continue
        except Exception as e:
            print(f"Error in PCA reduction: {e}")
            pass

    for name, (X_train_fs, _) in feature_sets.items():
        data_type = "CuPy" if isinstance(X_train_fs, cp.ndarray) else "NumPy"
        print(f"- {name}: {X_train_fs.shape[1]} features ({data_type})")

    print("\n--- model training and randomsearchcv: ---\n")
    models_to_tune = {
        'SVM': {
            # Replaced CUMLSVC with SklearnSVC (CPU-bound)
            'estimator': SklearnSVC(random_state=42),
            'param_grid': {
                'C': [0.1, 1, 10, 50],
                'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
                'kernel': ['rbf', 'linear']
            }
        },
        'LightGBM': {
            'estimator': lgb.LGBMClassifier(random_state=42, objective='binary', metric='binary_logloss', verbosity=-1, n_jobs=1, device='gpu'),
            'param_grid': {
                'n_estimators': [50, 100, 150],
                'learning_rate': [0.01, 0.05, 0.1],
                'max_depth': [-1, 10, 20],
                'num_leaves': [31, 50, 70],
                'subsample': [0.8, 0.9],
                'colsample_bytree': [0.8, 0.9, 1.0],
                'min_split_gain': [0.1],
                'min_child_samples': [5]
            }
        },
        'Custom_MLP': {
            'estimator': SklearnKerasClassifier(
                model=create_custom_mlp,
                loss=tf.keras.losses.BinaryCrossentropy(),
                epochs=100,
                batch_size=32,
                verbose=0,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, restore_best_weights=True)]
            ),
            'param_grid': {
                'model__hidden_layer_1_neurons': [64, 128, 256],
                'model__hidden_layer_2_neurons': [0, 64, 128],
                'model__dropout_rate': [0.2, 0.4, 0.6],
                'model__activation': ['relu', 'leaky_relu'],
                'optimizer__learning_rate': [0.001, 0.005, 0.01]
            }
        }
    }

    cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scoring_metric = 'f1'
    all_results = {}
    best_overall_test_score = -np.inf
    best_overall_combination = None
    best_overall_trained_model = None
    best_overall_X_test = None
    best_overall_transformer = None

    for model_name, model_config in models_to_tune.items():
        all_results[model_name] = {}
        estimator = model_config['estimator']
        param_distributions = model_config['param_grid']
        n_iter_search = model_config.get('n_iter', 8)
        print(f"\n\n=== train&tune {model_name} (Hybrid) ===\n")
        for fs_name in sorted(feature_sets.keys()):
            X_train_fs, X_test_fs = feature_sets[fs_name]
            fs_data_type = "CuPy" if isinstance(X_train_fs, cp.ndarray) else "NumPy"
            print(f"\n--- tune {model_name} on fs: {fs_name} ({X_train_fs.shape[1]} features, {fs_data_type}) ---\n")

            if X_train_fs is None or X_train_fs.shape[0] == 0: continue

            # Conditional n_jobs for CPU-bound models and large feature sets
            current_n_jobs = 4 # Default from config
            if isinstance(estimator, (SklearnSVC, SklearnLogisticRegression)) and X_train_fs.shape[1] > 10000: # Heuristic for "large"
                current_n_jobs = 1 # Force single-threaded for very high-dimensional CPU SVM/Logistic Regression to prevent OOM
                print(f"    WARNING: Setting n_jobs to {current_n_jobs} for {model_name} on {fs_name} due to high feature count for CPU model to prevent out of memory.\n")
            elif isinstance(estimator, (SklearnSVC, SklearnLogisticRegression)): # For smaller feature sets still use reasonable n_jobs
                current_n_jobs = -1 # Use all available CPU cores for CPU models with manageable feature count


            # Temporarily modify n_jobs in search_cv creation if needed
            # This requires recreating the RandomizedSearchCV object to set n_jobs dynamically
            if search_method == 'RandomSearch':
                search_cv = RandomizedSearchCV(
                   estimator=estimator,
                   param_distributions=param_distributions,
                   n_iter=n_iter_search,
                   cv=cv_strategy,
                   scoring=scoring_metric,
                   n_jobs=current_n_jobs, # Apply dynamic n_jobs here
                   verbose=1,
                   random_state=42
                )
            else: continue # Skip if search_method is not RandomSearch

            tuned_search = tune_model_hyperparameters(
                estimator,
                X_train_fs,
                y_train,
                param_grid=param_distributions,
                cv_strategy=cv_strategy,
                scoring=scoring_metric,
                search_method='RandomSearch', # This parameter is ignored if search_cv is created above
                n_iter=n_iter_search,         # This parameter is ignored if search_cv is created above
                validation_split_keras=0.2
            )
            
            # The previous tune_model_hyperparameters call creates and fits RandomizedSearchCV.
            # We need to ensure that the search_cv object used for fit has the correct n_jobs.
            # The current structure calls tune_model_hyperparameters which then creates search_cv.
            # We need to pass current_n_jobs into tune_model_hyperparameters if we want to control it there.
            # Let's refactor `tune_model_hyperparameters` to accept `n_jobs`.
            # For simplicity for now, let's directly create search_cv here in the loop as above.
            # The 'if tuned_search:' block is okay for flow.


            if tuned_search:
                best_model_for_combination = tuned_search.best_estimator_
                best_cv_score = tuned_search.best_score_
                best_params = tuned_search.best_params_
                print(f"best CV {scoring_metric} for {model_name} on {fs_name}: {best_cv_score:.4f}")
                test_metrics = evaluate_model(best_model_for_combination, X_test_fs, y_test, model_name, fs_name)
                all_results[model_name][fs_name] = {
                    'best_cv_score': best_cv_score,
                    'best_params': best_params,
                    'test_metrics': test_metrics,
                    'trained_model': best_model_for_combination,
                    'transformer': feature_transformers.get(fs_name)
                }
                if test_metrics and test_metrics.get('f1_score', -np.inf) > best_overall_test_score:
                    best_overall_test_score = test_metrics['f1_score']
                    best_overall_combination = (model_name, fs_name)
                    best_overall_trained_model = best_model_for_combination
                    best_overall_X_test = X_test_fs
                    best_overall_transformer = feature_transformers.get(fs_name)

    print("\n\n=== results summary for all models ===\n")
    if not all_results: pass
    else:
        print("\nbest cv f1 scores:\n")
        print("-------------------------------------------------\n")
        for model_name, fs_results in all_results.items():
            print(f"\n{model_name}:\n")
            if fs_results:
                for fs_name in sorted(fs_results.keys()):
                    result = fs_results[fs_name]
                    cv_score = result.get('best_cv_score', float('nan'))
                    print(f"  - {fs_name}: {cv_score:.4f}\n")
            else: pass

        print("\ntest results - f1:\n")
        print("----------------------------\n")
        best_f1_per_model = {}
        for model_name, fs_results in all_results.items():
            print(f"\n{model_name}:\n")
            if fs_results:
                best_test_f1_for_model = -np.inf
                best_fs_name_for_model = None

                for fs_name in sorted(fs_results.keys()):
                    result = fs_results[fs_name]
                    test_f1 = result.get('test_metrics', {}).get('f1_score', float('nan'))
                    print(f"  - {fs_name}: {test_f1:.4f}\n")
                    if not np.isnan(test_f1) and test_f1 > best_test_f1_for_model:
                        best_test_f1_for_model = test_f1
                        best_fs_name_for_model = fs_name
                if best_fs_name_for_model:
                    best_f1_per_model[model_name] = (best_fs_name_for_model, best_test_f1_for_model)
            else: continue

        print("\n=== best combo based on f1's ===\n")
        if best_overall_combination:
            model_name, fs_name = best_overall_combination
            best_result = all_results[model_name][fs_name]
            test_metrics = best_result['test_metrics']

            print(f"best model: {model_name}\n")
            actual_feature_count = feature_sets[fs_name][0].shape[1] if fs_name in feature_sets and feature_sets[fs_name][0] is not None else 'N/A'
            print(f"best fs: {fs_name} ({actual_feature_count} features)\n")
            print(f"best cvf1 sc: {best_result['best_cv_score']:.4f}\n")
            print(f"test f1: {test_metrics['f1_score']:.4f}\n")
            print(f"test acc: {test_metrics['accuracy']:.4f}\n")
            print(f"test prec: {test_metrics['precision']:.4f}\n")
            print(f"test rec: {test_metrics['recall']:.4f}\n")
            print(f"params: {best_result['best_params']}\n")
            print(f"conf.m.:\n{np.array(test_metrics['confusion_matrix'])}\n")
        else: pass

    MODEL_SAVE_DIR = "/mnt/c/Users/BerenÜnveren/Desktop/YAP470/models/"
    os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
    print("\n--- saving best hybrids ---\n")
    if 'best_f1_per_model' not in locals() or not best_f1_per_model: pass
    else:
        for model_name, (best_fs_name_for_model, best_test_f1_for_model) in best_f1_per_model.items():
            print(f"\nprocessing {model_name}...\n")
            if best_fs_name_for_model and model_name in all_results and best_fs_name_for_model in all_results[model_name]:
                best_combination_results = all_results[model_name][best_fs_name_for_model]
                model_to_save = best_combination_results.get('trained_model')
                transformer_to_save = best_combination_results.get('transformer')
                if model_to_save:
                    is_keras_model = isinstance(model_to_save, KerasClassifier)
                    file_extension = '.keras' if is_keras_model else '.pkl'
                    model_filename = f'Dfire_hybrid_{model_name.lower()}_best_model_{best_fs_name_for_model}{file_extension}'
                    MODEL_SAVE_PATH_ALG = os.path.join(MODEL_SAVE_DIR, model_filename)
                    try:
                        if is_keras_model: model_to_save.model_.save(MODEL_SAVE_PATH_ALG)
                        else: joblib.dump(model_to_save, MODEL_SAVE_PATH_ALG)
                        print(f"   saved model: {MODEL_SAVE_PATH_ALG}\n")
                    except Exception as e:
                        print(f"   Error saving model {model_filename}: {e}\n")
                else:
                    print(f"   No trained model found for {model_name} on {best_fs_name_for_model}.\n")

                if transformer_to_save and 'All_CNN' not in best_fs_name_for_model:
                     transformer_filename = f'Dfire_hybrid_transformer_{best_fs_name_for_model}.pkl'
                     TRANSFORMER_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, transformer_filename)
                     try:
                         joblib.dump(transformer_to_save, TRANSFORMER_SAVE_PATH)
                         print(f"   saved feature s/r transformer: {TRANSFORMER_SAVE_PATH}\n")
                     except Exception as e:
                         print(f"   Error saving transformer {transformer_filename}: {e}\n")
                         print("   (Note: Saving cuML transformers with joblib might require specific handling or conversion to CPU state for complex objects.)\n")

    print("\n--- all done!! ---\n")

2025-07-23 21:26:27.381725: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-23 21:26:29.657305: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to us


--- extracting cnn features: ---


2025-07-23 21:28:20.378420: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-23 21:28:20.402094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-23 21:28:20.402496: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-23 21:28:20.414822: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-23 21:28:20.415005: I external/local_xla/xla/stream_executor

training features shape: (1221, 65536)
testing features shape: (408, 65536)
training labels shape: (1221,)
testing labels shape: (408,)

--- scaling cnn features (GPU accelerated with cuML StandardScaler): ---

--- selection & pca: ---

corr selection: 75% (CPU-bound SelectKBest)...

original feature shape: (1221, 65536)
selected feature shape: (1221, 49152)

corr selection: 50% (CPU-bound SelectKBest)...

original feature shape: (1221, 65536)
selected feature shape: (1221, 32768)

rfe selection with 75% (step=0.1, CPU-bound estimator with scikit-learn LogisticRegression)...

original feature shape: (1221, 65536)
selected feature shape: (1221, 49152)

rfe selection with 50% (step=0.1, CPU-bound estimator with scikit-learn LogisticRegression)...

original feature shape: (1221, 65536)
selected feature shape: (1221, 32768)

pca with n_components=0.85 (GPU accelerated with cuML PCA)...

PCA error: exception occurred! file=/opt/conda/conda-bld/work/cpp/src/pca/pca.cuh line=102: Parameter n_

NameError: name 'search_method' is not defined